Dados Sintéticos
=============
Convolve um espalhador com um refletor

---

In [ ]:
from __future__ import print_function
import numpy as np
from scipy.signal import convolve2d
import matplotlib.pyplot as plt

# Importa bibliotecas próprias
from utils import * 

# Configura o matplolib para plotar inline
%matplotlib qt5
#%matplotlib notebook

---
Importa dado de reflexão migrado em tempo e respectivas estruturas

In [ ]:
# Definição de arquivos de entrada
reflection_file = './reflexoes_migradas/solimoes_migrado.sgy'
edges_file = './estruturas/solimoes_edges.pickle'

color = 'seismic'
plot_bar = True
clip = 0.3

# Carrega dados de reflexão
if not reflection_file is None: 
    traces = load_traces(reflection_file)
    reflection_image = get_data_from_traces(traces)
    print("Dimensões do dado 1:", reflection_image.shape)
    print()
    
# Carrega arestas
edges = load_pickle(edges_file)['edges']
print("Estruturas:", edges.shape)



In [ ]:
# Visualiza dados

color = 'seismic'
plot_bar = True
clip = 1.0

multiple_event_viewer(reflection_image, edges,
             cdp_offset=traces[0].cdp, dt=traces[0].dt,
             vmin2=0.0, vmax2=1.0, color2='gray',
             clip_percent1=clip, color1=color)

---
Importa PSFs e seleciona uma janela para gerar o dado

In [ ]:
# Arquivo com PSFs
psf_file = './janelas/jequitinhonha.pickle'

data_dic = load_pickle(psf_file)
scatters = data_dic['scatters']
frequencies = data_dic['frequencies']
scatter_widths = data_dic['scatter_widths']

In [ ]:
# Visualiza PSFs importados
max_samples = 30
color='seismic'
resize=3.5

# Cria título informativo para cada janela
labels = ['freq:%.2f, width:%d' % (f,w) for f,w in zip(frequencies, scatter_widths)]

if max_samples < scatters.shape[0]:
    num_samples = max_samples
else:
    num_samples = None
    
view_samples(scatters, labels, num_samples=num_samples, resize=resize, color=color)

In [ ]:
# Seleciona um PSF para convolver com o dado de estruturas
psf_index = 3
color='seismic'
resize=10.0

psf = scatters[psf_index]
psf_freq = frequencies[psf_index]
psf_width = scatter_widths[psf_index]

title = 'freq:%.2f, width:%d' % (psf_freq, psf_width)

multiple_event_viewer(psf, color1=color, resize=resize, title1=title)


---
Gera um 'ponto' a partir de uma wavelet de Ricker com a mesma frequência e largura que o PSF escolhido

In [ ]:
taper_size = int(psf_width)
taper_size = 4
window_size = psf.shape[0]
dt = 0.004

# Constrói wavelet
wavelet = ricker_wavelet(psf_freq, window_size, dt)
# Constrói wavelet 2D
wavelet = wavelet.reshape((1,-1)).T
wavelet_2D = np.empty((window_size, taper_size*2+1), dtype=np.float32)
wavelet_2D[:,:] = wavelet
# Suaviza bordas aplicando função de taper
wavelet_2D = apply_taper(wavelet_2D, taper_size)
print("Wavelet_2D", wavelet_2D.shape)

In [ ]:
# Constrói janela para visualização equivalente ao PSF
color='seismic'
resize=10.0

title1='PSF'
title2='Ricker 2D'
    
center = window_size/2
wavelet_window = np.zeros(psf.shape)
wavelet_window[:, center-taper_size:center+(taper_size+1)] += wavelet_2D

multiple_event_viewer(psf, wavelet_window, resize=resize, color1=color, color2=color, dt=dt,
                     title1=title1, title2=title2)

---
Cria dados sintéticos convolvendo PSF, wavelet e wavelet 2D com o dado de bordas

In [ ]:
noise = 0.1
background = (np.random.random(edges.shape) - 0.5) * noise
noisy_edges = background + edges

synthetic_psf = convolve2d(noisy_edges, psf, mode='same')
synthetic_ricker = convolve2d(noisy_edges, wavelet, mode='same')
synthetic_ricker2D = convolve2d(noisy_edges, wavelet_2D, mode='same')

In [ ]:
# Compara dado original com convolvido com PSF
color = 'seismic'
clip = 1.0

title1 = 'Dado original'
title2 = 'Convolvido PSF'

multiple_event_viewer(reflection_image, synthetic_psf,
                     color1=color, color2=color,
                     clip_percent1=clip, clip_percent2=clip,
                     title1=title1, title2=title2)

In [ ]:
# Compara convolvido com PSF, wavelet e wavelet2D
title1 = 'Convolvido PSF'
title2 = 'Convolvido Ricker 2D'
title3 = 'Convolvido Ricker'

color = 'seismic'

multiple_event_viewer(synthetic_psf, synthetic_ricker2D, synthetic_ricker,
                     color1=color, color2=color, color3=color,
                     title1=title1, title2=title2, title3=title3)

In [ ]:
# Compara convolvido com PSF, wavelet e wavelet2D
title1 = 'Refletores'
title2 = 'Convolvido PSF'
title3 = 'Convolvido Ricker 2D'

color = 'seismic'

resize = 7.0

y_begin, x_begin = picks_to_index([(1303.835223, 1.540451)], traces)[0]
y_end, x_end = picks_to_index([(1534.616626, 1.849841)], traces)[0]

vmax = np.max(np.abs(synthetic_psf))
synthetic_psf /= vmax
vmax = np.max(np.abs(synthetic_ricker))
synthetic_ricker /= vmax

multiple_event_viewer(edges, synthetic_psf, synthetic_ricker2D,
                      x_begin=x_begin, x_end=x_end, y_begin=y_begin, y_end=y_end,
                      cdp_offset=traces[0].cdp, dt=traces[0].dt,
                     vmin1=0.0, vmax1=1.0,
                     resize=resize, color1='gray', color2=color, color3=color,
                     #bar1=False, bar2=False, bar3=False,
                     title1=title1, title2=title2, title3=title3)